# Project 7

## Sys Ops

In [1]:
cd ..

/home/alex/tmp/P7-ML-Rudy


/home/alex/tmp/P7-ML-Rudy/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
pwd

'/home/alex/tmp/P7-ML-Rudy'

## Imports 

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import missingno as msno

from sklearn.feature_selection import *
from sklearn.feature_extraction import *
from sklearn.decomposition import *
from sklearn.model_selection import *
from sklearn.impute import *
from sklearn.preprocessing import *
from sklearn.pipeline import *
from sklearn.covariance import *
from sklearn.metrics import *
from sklearn.compose import *

from sklearn.dummy import *
from sklearn.linear_model import *
from sklearn.neighbors import *
from sklearn.svm import *
from sklearn.ensemble import *
from sklearn.cluster import *

In [4]:
# DO NOT REMOVE THIS LINE

## Data

In [5]:
fn = "application_train.csv"
df = pd.read_csv("./data/source/" + fn)

In [ ]:
df

In [ ]:
df = df.sample(frac = 1)
df

In [ ]:
# filter for correl
#df = df.filter(['TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
#                'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH', 'DAYS_EMPLOYED'
#                ])

df = df.filter(['TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
                'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
                'ORGANIZATION_TYPE', 'OCCUPATION_TYPE', 'EXT_SOURCE_1', 'EXT_SOURCE_2',  'EXT_SOURCE_3'
                ])


df

## EDA Exploratory Data Analysis

In [ ]:
df.sort_values('AMT_INCOME_TOTAL', ascending = False)

In [ ]:
df.sort_values('AMT_CREDIT', ascending = False)

In [ ]:
df.sort_values('AMT_ANNUITY', ascending = False)

In [ ]:
df.sort_values('AMT_GOODS_PRICE', ascending = False)

In [ ]:
df.sort_values('DAYS_BIRTH', ascending = False)

In [ ]:
df.sort_values('DAYS_EMPLOYED', ascending = False)

In [ ]:
df.loc[df['DAYS_EMPLOYED'] == 365243, 'DAYS_EMPLOYED'] = float('NaN')

In [ ]:
df.sort_values('DAYS_EMPLOYED', ascending = False)

In [ ]:
df.sort_values('DAYS_EMPLOYED', ascending = True)

In [ ]:
df.loc[df['ORGANIZATION_TYPE'] == 'XNA', 'ORGANIZATION_TYPE'] = float('NaN')

In [ ]:
df.TARGET.value_counts(normalize=True)

In [ ]:
#Transformations

df.loc[df['CODE_GENDER'] == 'M', 'CODE_GENDER'] = 1
df.loc[df['CODE_GENDER'] == 'F', 'CODE_GENDER'] = 2
df.loc[df['CODE_GENDER'] == 'XNA', 'CODE_GENDER'] = 2

df.loc[df['FLAG_OWN_CAR'] == 'Y', 'FLAG_OWN_CAR'] = 1
df.loc[df['FLAG_OWN_CAR'] == 'N', 'FLAG_OWN_CAR'] = 0

df.loc[df['FLAG_OWN_REALTY'] == 'Y', 'FLAG_OWN_REALTY'] = 1
df.loc[df['FLAG_OWN_REALTY'] == 'N', 'FLAG_OWN_REALTY'] = 0

df.DAYS_BIRTH = - df.DAYS_BIRTH / 365
df.DAYS_EMPLOYED = - df.DAYS_EMPLOYED / 365

df

In [ ]:
df.isna().sum()

In [ ]:
#df.corr()

In [ ]:


corr = df.select_dtypes(include=np.number).corr()
mask = np.triu(corr)
sns.heatmap(corr, annot=True, vmin=-1, vmax=1, cmap="coolwarm", fmt=".2f", mask=mask)

In [ ]:
# on construit la matrice X des Features
X = df.drop(columns='TARGET')
X

In [ ]:
# On construit l vecteur y des Targets
y = df['TARGET']
y

In [ ]:
y.values

In [ ]:
def resultize(grid, verbose=2):
    """make results based on gridsearchcv"""
    
    #make df
    results = pd.DataFrame(grid.cv_results_)
    
    #clean columns
    cols = [i for i in results.columns if not i.startswith("split")]
    results = results.loc[:, cols]
    
    if verbose==2:
        print("coucou")
        
    return results.round(2)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

## Modelisation

### Refactoring avec utilisation de Pipeline 

### Création du ColumnTransformer

In [ ]:
numerical_features = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
                'AMT_GOODS_PRICE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
                'EXT_SOURCE_1', 'EXT_SOURCE_2',  'EXT_SOURCE_3']

categorical_features = ['NAME_CONTRACT_TYPE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 
                        'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'ORGANIZATION_TYPE']

In [ ]:
# Création de 2 Pipelines

numerical_pipeline = make_pipeline(KNNImputer(),
                                   StandardScaler())

categorical_pipeline = make_pipeline(SimpleImputer(strategy = 'most_frequent'),
                                     OneHotEncoder())

In [ ]:
# Création du ColumnTransformer
# numerical_pipeline est appliquée sur les numerical_features
# categorical_pipeline est appliquée sur les categorical_features

ColumnTransformer = make_column_transformer((numerical_pipeline, numerical_features),
                                            (categorical_pipeline, categorical_features))
ColumnTransformer

In [ ]:
pipe = Pipeline([("columntransformer", ColumnTransformer),
                #  ("inputer", KNNImputer()),
                #  ("scaler", StandardScaler()),
                 ("estimator", DummyRegressor()),
                ])

In [ ]:
grid2 = GridSearchCV(pipe, param_grid={}, cv=5, verbose=2, return_train_score=True, n_jobs=-1)
grid2

In [ ]:
grid2.fit(X, y)

In [ ]:
resultize(grid2)